In [62]:
import string
import pandas as pd
from unidecode import unidecode
import re
from random import randint
import requests
import numpy as np
import os
import csv
import uuid

In [63]:
def save_dataframe(df, filename):
    # Verifica se o diretório existe e cria caso não exista
    diretorio = './arquivos_para_carga'
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)
    
    # Salva o dataframe em um arquivo csv no diretório criado
    filepath = f'./{diretorio}/{filename}.csv'
    df.to_csv(filepath, index=False, sep=';')

In [64]:
# definir parâmetros do arquivo CSV
csv_kwargs = {'index': False,
              'sep': ';',
              'quoting': csv.QUOTE_NONE,
              'quotechar': '"'}

In [65]:
df_contribuintes_receitas = pd.read_csv('arquivos_para_carga/dados_contribuintes_receita.csv', sep=';')
df_contribuintes_receitas.head()

,CPF/CNPJ,CONTRIBUINTE,TIPO,CODIGO
0,08.923.971/0001-15,prefeitura municipal de cajazeiras,prefeitura municipal,886cd0eabf5a18
1,038.628.134-36,joao jose de andrade sobrinho,prefeitura municipal,af7e52bc3ee1a8
2,911.164.224-68,clara maria honorato,prefeitura municipal,09dace633966e6
3,070.405.144-35,fopag creas i,prefeitura municipal,2ec12bef7667c1
4,00.448.994/0001-03,classic viagens e turismo eireli me,prefeitura municipal,eefb6011a40df6


In [66]:
df_contribuintes_despesas = pd.read_csv('arquivos_para_carga/dados_contribuintes_despesas.csv', sep=';')
df_contribuintes_despesas.head()

,CPF/CNPJ,CONTRIBUINTE,TIPO,CONCAT_CODIGO,CODIGO
0,09.123.654/0001-87,cagepacom de agua e esgotos da paraiba,prefeitura municipal,09.123.654/0001-87-1,86eabac73f48c4
1,080.329.514-66,raniely duarte barreto,prefeitura municipal,080.329.514-66-1,c86a88163a502f
2,025.419.924-08,cicera de freitas rocha,prefeitura municipal,025.419.924-08-1,0d5215bcd76d11
3,526.406.044-49,maria rivonia pereira de oliveira,prefeitura municipal,526.406.044-49-1,6709a10d3ece82
4,823.237.713-53,luciana barreto rolim,prefeitura municipal,823.237.713-53-1,a4c224f5451c8e


In [67]:
df_contribuintes = pd.concat([df_contribuintes_receitas, df_contribuintes_despesas]).reset_index(drop=True)
df_contribuintes

,CPF/CNPJ,CONTRIBUINTE,TIPO,CODIGO,CONCAT_CODIGO
0,08.923.971/0001-15,prefeitura municipal de cajazeiras,prefeitura municipal,886cd0eabf5a18,NaN
1,038.628.134-36,joao jose de andrade sobrinho,prefeitura municipal,af7e52bc3ee1a8,NaN
2,911.164.224-68,clara maria honorato,prefeitura municipal,09dace633966e6,NaN
3,070.405.144-35,fopag creas i,prefeitura municipal,2ec12bef7667c1,NaN
4,00.448.994/0001-03,classic viagens e turismo eireli me,prefeitura municipal,eefb6011a40df6,NaN
...,...,...,...,...,...
13030,102.097.424-96,prefeitura municipal de cajazeiras,camara municipal,2521c02e3c6b10,102.097.424-96-4
13031,08.923.971/0001-15,prefeitura municipal de cajazeiras,camara municipal,6e8245200e6967,08.923.971/0001-15-4
13032,646.603.624-34,marcos helder,camara municipal,a50e69b97e2b34,646.603.624-34-4
13033,08.841.553/0001-89,camara municipal de cajazeiras,camara municipal,24bf0d009e9239,08.841.553/0001-89-4


In [68]:
df_contribuintes['CODIGO'].value_counts()

886cd0eabf5a18    2
9a1670723686fc    2
f93cae8ab439a7    2
a8fa73e96f73ed    2
7b52070f971b04    2
                 ..
970335af9cc22e    1
320b5146cc5206    1
fae31f72db4e67    1
fa388f00027307    1
2c62524621c1dc    1
Name: CODIGO, Length: 6723, dtype: int64

In [69]:
df_contribuintes.drop_duplicates(subset='CODIGO', inplace=True)
df_contribuintes.reset_index(drop=True)

,CPF/CNPJ,CONTRIBUINTE,TIPO,CODIGO,CONCAT_CODIGO
0,08.923.971/0001-15,prefeitura municipal de cajazeiras,prefeitura municipal,886cd0eabf5a18,NaN
1,038.628.134-36,joao jose de andrade sobrinho,prefeitura municipal,af7e52bc3ee1a8,NaN
2,911.164.224-68,clara maria honorato,prefeitura municipal,09dace633966e6,NaN
3,070.405.144-35,fopag creas i,prefeitura municipal,2ec12bef7667c1,NaN
4,00.448.994/0001-03,classic viagens e turismo eireli me,prefeitura municipal,eefb6011a40df6,NaN
...,...,...,...,...,...
6718,102.097.424-96,prefeitura municipal de cajazeiras,camara municipal,2521c02e3c6b10,NaN
6719,08.923.971/0001-15,prefeitura municipal de cajazeiras,camara municipal,6e8245200e6967,NaN
6720,646.603.624-34,marcos helder,camara municipal,a50e69b97e2b34,NaN
6721,08.841.553/0001-89,camara municipal de cajazeiras,camara municipal,24bf0d009e9239,NaN


In [70]:
df_contribuintes['CODIGO'].value_counts()

886cd0eabf5a18    1
9a1670723686fc    1
f93cae8ab439a7    1
a8fa73e96f73ed    1
7b52070f971b04    1
                 ..
89bb3c001b3bc4    1
ed3138ace9e7af    1
d2c8968510c728    1
0328e99b641b71    1
57efb89539a8a0    1
Name: CODIGO, Length: 6723, dtype: int64

In [71]:
df_contribuintes.drop(['CONCAT_CODIGO'], axis=1, inplace=True)
df_contribuintes.head()

,CPF/CNPJ,CONTRIBUINTE,TIPO,CODIGO
0,08.923.971/0001-15,prefeitura municipal de cajazeiras,prefeitura municipal,886cd0eabf5a18
1,038.628.134-36,joao jose de andrade sobrinho,prefeitura municipal,af7e52bc3ee1a8
2,911.164.224-68,clara maria honorato,prefeitura municipal,09dace633966e6
3,070.405.144-35,fopag creas i,prefeitura municipal,2ec12bef7667c1
4,00.448.994/0001-03,classic viagens e turismo eireli me,prefeitura municipal,eefb6011a40df6


In [72]:
dic_fonte_dados = {1:'prefeitura municipal',
                   2:'instituto de previdencia e assistencia social',
                   3:'sctrans',
                   4:'camara municipal'}

In [73]:
df_contribuintes_atualizado = df_contribuintes.copy()
df_contribuintes_atualizado['TIPO'] = df_contribuintes_atualizado['TIPO'].replace(dic_fonte_dados)
df_contribuintes_atualizado['TIPO'].value_counts()

prefeitura municipal                             5957
sctrans                                           398
instituto de previdencia e assistencia social     363
camara municipal                                    5
Name: TIPO, dtype: int64

In [74]:
save_dataframe(df_contribuintes_atualizado,'dados_contribuintes')

with open('arquivos_para_carga/dados_contribuintes_receita.csv', 'w', encoding='utf-8', newline='') as f:
    df_contribuintes_atualizado.to_csv(f, **csv_kwargs)

# Tratamento das classes de receita e despesa

In [75]:
df_classe_receita = pd.read_csv('C:/Users/allys/Dados_TCC/dw_contas_publicas_2022/TCC2/dados/01_receitas/dados_para_carga/dados_tipos_receitas.csv', sep=';')
df_classe_receita.rename({'RECEITA':'DESCRICAO'}, axis=1, inplace=True)
df_classe_receita.head()

,CODIGO,DESCRICAO
0,1722010101,cotaparte do icms
1,1721010201,cotaparte do fundo de participacao dos municipios
2,9722010101,deducao de receita para a formacao do fundef icms
3,9721010201,deducao de receita do fpm fundef e redutor fin...
4,1113050002,sna simples nacional


In [76]:
df_classe_despesa = pd.read_csv('arquivos_para_carga/dados_tipo_despesas.csv', sep=';')
df_classe_despesa.rename({'COD_DESPESA':'CODIGO','TIPO_DESPESA':'DESCRICAO'}, axis=1, inplace=True)
df_classe_despesa.head()

,CODIGO,DESCRICAO
0,339039,outros servicos de terceiros pessoa juridica
1,319004,contratacao por tempo determinado
2,319011,vencimentos e vantagens fixas pessoal civil
3,339030,material de consumo
4,335041,contribuicoes


In [77]:
df_codigos_descricoes_tipo_receita_despesa = pd.concat([df_classe_receita, df_classe_despesa])
df_codigos_descricoes_tipo_receita_despesa

,CODIGO,DESCRICAO
0,1722010101,cotaparte do icms
1,1721010201,cotaparte do fundo de participacao dos municipios
2,9722010101,deducao de receita para a formacao do fundef icms
3,9721010201,deducao de receita do fpm fundef e redutor fin...
4,1113050002,sna simples nacional
...,...,...
31,319003,pensoes do rpps e do militar
32,319001,aposentadorias do rpps reserva remunerada e re...
33,335043,subvencoes sociais
34,339031,premiacoes culturais artisticas cientificas de...


In [78]:
df_codigos_descricoes_tipo_receita_despesa['CODIGO'] = df_codigos_descricoes_tipo_receita_despesa['CODIGO'].astype(str)
df_codigos_descricoes_tipo_receita_despesa['CODIGO'] = df_codigos_descricoes_tipo_receita_despesa['CODIGO'].replace('1','000001')
df_codigos_descricoes_tipo_receita_despesa.reset_index(drop=True, inplace=True)
df_codigos_descricoes_tipo_receita_despesa


,CODIGO,DESCRICAO
0,1722010101,cotaparte do icms
1,1721010201,cotaparte do fundo de participacao dos municipios
2,9722010101,deducao de receita para a formacao do fundef icms
3,9721010201,deducao de receita do fpm fundef e redutor fin...
4,1113050002,sna simples nacional
...,...,...
447,319003,pensoes do rpps e do militar
448,319001,aposentadorias do rpps reserva remunerada e re...
449,335043,subvencoes sociais
450,339031,premiacoes culturais artisticas cientificas de...


In [79]:
save_dataframe(df_codigos_descricoes_tipo_receita_despesa,'dados_classes_receitas_despeas')

with open('arquivos_para_carga/dados_classes_receitas_despeas.csv', 'w', encoding='utf-8', newline='') as f:
    df_codigos_descricoes_tipo_receita_despesa.to_csv(f, **csv_kwargs)